# Counting objects workflow (batch)

---
*Introduction to Image Analysis Workshop*

*Stefania Marcotti (stefania.marcotti@crick.ac.uk)*

*Intro to building batch image analysis pipelines with Python*

*CC-BY-SA-4.0 license: creativecommons.org/licenses/by-sa/4.0/*

---

Following on from the previous [notebook](https://github.com/RMS-DAIM/introduction-to-image-analysis/tree/main/Scripts/Jupyter/counting_objects.ipynb), we run the same pipeline on all the images available in the [`Data/idr0028`](https://github.com/RMS-DAIM/introduction-to-image-analysis/tree/main/Data/idr0028) folder.

### Import libraries

Note we are importing also the library `os` (API [here](https://docs.python.org/3/library/os.html)) which provides miscellaneous operating system interfaces.

In [ ]:
import os

import numpy as np

from skimage import io
from skimage import filters
from skimage import measure

import matplotlib.pyplot as plt

import pandas as pd

### Import data
This time we are reading all of the files in the folder - note the use of the wildcard `*`!

In [ ]:
# get the list of all files and directories
path = '../../Data/idr0028/'
dir_list = os.listdir(path)

print(*dir_list, sep='\n')

In [ ]:
# read all .tif files in folder
im_read = io.imread('../../Data/idr0028/*.tif')

In [ ]:
# check the dimensions of the object
print('Object dimensions:', im_read.shape)

The image dimensions are currently in the (n, y, x, c) format. Ideally, by convention, we would want the dimensions of the image to be (n, c, x, y). We can rearrange the dimensions by using the numpy function `transpose`.

In [ ]:
im_all = np.transpose(im_read, (0, 3, 1, 2))
print('New object dimensions:', im_all.shape)

We can visualise some example nuclear images by running the cell below. The variable `im_idx` indicates which image we want to open (first=1, second=2, ...)

In [ ]:
# select image
im_idx = 0
im = im_all[im_idx,0,]

# display the chosen image
fig, ax = plt.subplots(figsize=(4,3))
ax.imshow(im, cmap='gray')
ax.axis('off')
plt.tight_layout()

### Perform the analysis on one image at the time
By using a `for` loop, we can open one image at the time, and perform the same exact pipeline as before to count the objects and measure their area and eccentricity. We can then append the results in a long list where each cell is an n=1 and plot the output.

In [ ]:
# initialise output
obj_count = []
props_df = pd.DataFrame()

# set up for loop for all the images available in the folder
for im_idx in range(im_all.shape[0]):
    # open image (only ch0)
    im = im_all[im_idx,0,]
    # filter with Gaussian
    im_gauss = filters.gaussian(im, sigma=5)
    # threshold with Otsu
    thresh = filters.threshold_otsu(im_gauss)
    im_thresh = im_gauss >= thresh
    # label mask
    labels = measure.label(im_thresh)
    # count objects
    obj_count = np.append(obj_count,labels.max())
    # measure properties
    props = measure.regionprops_table(labels, im, properties=['area', 'eccentricity'])
    props = pd.DataFrame(props)
    # add image ID and object ID
    props['image_ID'] = dir_list[im_idx]
    props['object_ID'] = props.index+1
    # add to output dataframe
    props_df = pd.concat([props_df, pd.DataFrame(props)], ignore_index=True)

In [ ]:
# print output [obj_count]
print('The number of nuclei for each image is:', obj_count)

In [ ]:
for im_idx in range(im_all.shape[0]):
    print('The number of nuclei in', dir_list[im_idx], 'is', obj_count[im_idx])

In [ ]:
# show the beginning of the properties dataframe
props_df.head()

In [ ]:
# how many nuclei did we analyse?
print('We analysed', len(props_df), 'nuclei in', im_all.shape[0], 'images')

### Plot some results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(4,3))

axs[0].boxplot(props_df['area'])
axs[0].set_title('Nuclear area (px)')

axs[1].boxplot(props_df['eccentricity'])
axs[1].set_title('Nuclear eccentricity')

plt.tight_layout()

### Save outputs

In [ ]:
# save dataframe as excel file
props_df.to_excel('../../Data/idr0028/results_jupyter.xlsx')

In [ ]:
# save label images
for im_idx in range(im_all.shape[0]):
    # open image (only ch0)
    im = im_all[im_idx,0,]
    # filter with Gaussian
    im_gauss = filters.gaussian(im, sigma=5)
    # threshold with Otsu
    thresh = filters.threshold_otsu(im_gauss)
    im_thresh = im_gauss >= thresh
    # label mask
    labels = measure.label(im_thresh)
    # save
    file_name = '../../Data/idr0028/'+dir_list[im_idx][0:-4]+'_labels.tif'
    io.imsave(file_name, labels)

### Visualise the data using `napari`

You might want to check the import options at [this `napari` API documentation page](https://napari.org/stable/api/napari.html#napari.view_image)

In [ ]:
im_idx = 0
# open image (only ch0)
im = im_all[im_idx,0,]
# filter with Gaussian
im_gauss = filters.gaussian(im, sigma=5)
# threshold with Otsu
thresh = filters.threshold_otsu(im_gauss)
im_thresh = im_gauss >= thresh
# label mask
labels = measure.label(im_thresh)

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()
viewer.add_image(im_all[im_idx,0,], name='nuclei', colormap='cyan', blending='additive')
viewer.add_image(im_all[im_idx,1,], name='tubulin', colormap='magenta', blending='additive')
viewer.add_image(im_all[im_idx,2,], name='actin', colormap='green', blending='additive')
viewer.add_image(im_all[im_idx,3,], name='YAP/TAZ', colormap='yellow', blending='additive')

In [ ]:
viewer = napari.Viewer()
viewer.add_image(im, name='nuclei', blending='additive')
viewer.add_image(im_gauss, name='nuclei_gauss', blending='additive')
viewer.add_labels(im_thresh, name='nuclei_thresh', blending='additive')
viewer.add_labels(labels, name='nuclei_label', blending='additive')

### Further reading
If you're interested in knowing more about image analysis in Python, there are a lot of free resources out there! One good starting point is the [IAFIG-RMS Python for Bioimage Analysis Course](https://github.com/RMS-DAIM/Python-for-Bioimage-Analysis). Good luck with your image analysis adventures!